In [1]:
# Step 2: Import necessary libraries
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only



In [2]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Load the MNIST dataset
(x_all, y_all), _ = keras.datasets.mnist.load_data()

# Flatten and normalize the images
x_all = x_all.reshape((x_all.shape[0], -1)).astype("float32") / 255.0

# Split into train+val and test (80/20)
x_temp, x_test, y_temp, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

# Split train+val into train and validation (75/25 of 80% = 60/20 overall)
x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)

In [3]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


In [5]:
# define model with hyperparameters

def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [6]:
# Create a RandomSearch Tuner 

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space 
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [7]:
# Run the hyperparameter search 
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

# Display a summary of the results 
tuner.results_summary() 

Trial 10 Complete [00h 03m 53s]
val_accuracy: 0.971450001001358

Best val_accuracy So Far: 0.9793500006198883
Total elapsed time: 00h 34m 39s
Results summary
Results in my_dir\intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 02 summary
Hyperparameters:
units: 320
learning_rate: 0.0011996856608783143
Score: 0.9793500006198883

Trial 00 summary
Hyperparameters:
units: 288
learning_rate: 0.00039679444080165674
Score: 0.9775999784469604

Trial 07 summary
Hyperparameters:
units: 384
learning_rate: 0.003325045530269275
Score: 0.976500004529953

Trial 05 summary
Hyperparameters:
units: 192
learning_rate: 0.0002915350155652438
Score: 0.9728499948978424

Trial 03 summary
Hyperparameters:
units: 320
learning_rate: 0.0002045623767248638
Score: 0.9727999866008759

Trial 04 summary
Hyperparameters:
units: 64
learning_rate: 0.002511502810092802
Score: 0.9727500081062317

Trial 09 summary
Hyperparameters:
units: 512
learning_rate: 0.004484875866260992
Score: 0

In [ ]:
# Retrieve the best hyperparameters 

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
print(f""" 

The optimal number of units in the first dense layer is {best_hps.get('units')}. 

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 

""") 

# Step 2: Build and Train the Model with Best Hyperparameters 
model = tuner.hypermodel.build(best_hps) 
model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

# Evaluate the model on the test set 
test_loss, test_acc = model.evaluate(x_val, y_val) 
print(f'Test accuracy: {test_acc}') 